In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from prophet import Prophet
from sklearn.metrics import mean_absolute_error

In [ ]:
#df=pd.read_csv('JB_DATA.csv')

In [ ]:
'''pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) '''

"pd.set_option('display.max_rows', None)\npd.set_option('display.max_columns', None) "

In [ ]:
#df.head()

In [ ]:
#df.shape

In [ ]:
#df.columns

In [ ]:
#df.isna().sum()

In [ ]:
#df.to_csv('sales_data.csv', index=False)

In [ ]:
df=pd.read_csv('sales_data.csv')

In [ ]:
df.head()

,# created_date,id,created_by,created_name,transaction_branch_id,branch_name,branch_email,branch_type,transaction_type_id,member_card,...,taxable_amount,TAX_AMOUNT,GST_AMOUNT,SGST_AMOUNT,CGST_AMOUNT,IGST_AMOUNT,Message_Status,Membership_expiry_date,TAX_STATE,User_Name
0,06-07-2024,41571158,39977,FRONTOFFICE ELC,131,Electronic City Phase 1,frontoffice@elc1.justbooksclc.com,Active User Branch,4,M227544,...,100,18.00,18.00,9.00,9.00,0.0,0,17-09-2024,KARNATAKA,avinashmodaliyar1
1,06-07-2024,41571159,10127,FRONTOFFICE VBL,26,Vijaya Bank Layout,sreejaya.shenoy@vij.justbooksclc.com,Profit share,2,M177663,...,300,54.00,54.00,27.00,27.00,0.0,0,06-08-2024,KARNATAKA,blrdrmembers
2,06-07-2024,41571160,39977,FRONTOFFICE ELC,131,Electronic City Phase 1,frontoffice@elc1.justbooksclc.com,Active User Branch,2,M200974,...,3840,691.17,691.17,345.58,345.58,0.0,0,06-07-2025,KARNATAKA,avinashmodaliyar1
3,06-07-2024,41571161,39966,FRONT OFFICE HSR,126,HSR Layout,frontoffice@hsr.justbooksclc.com,Active User Branch,1,M227899,...,2159,388.68,388.68,194.34,194.34,0.0,0,06-01-2025,KARNATAKA,avinashmodaliyar1
4,06-07-2024,41571162,40016,SRIVALLI HN,108,Nagarabhavi BDA Complex,frontoffice@kng.justbooksclc.com,Profit share,2,M226803,...,1200,216.00,216.00,108.00,108.00,0.0,0,06-10-2024,KARNATAKA,blrdrmembers


In [ ]:
columns_keep= ['# created_date', 'id', 'created_by', 'transaction_branch_id', 'transaction_type_id',
               'amount_paid', 'Coupon_Discount', 'magazine_fee_paid', 'payable_amount',
               'reading_fee_paid', 'reversed', 'over_due_adjustment_amount', 'last_card_number',
               'primus_amount', 'adjustment_amount', 'security_deposit', 'Percentage_Share',
               'subscription_id', 'member_branch_id', 'transaction', 'Transaction_Month', 'Transaction_Year',
               'created_name', 'branch_name', 'branch_type', 'member_card', 'Member_Name', 'email',
               'Number_of_Months', 'display_name', 'Renewal_Amount', 'Message_Status', 'Membership_expiry_date', 'User_Name']
df_fin=df[columns_keep]

In [ ]:
df_fin['created_date'] = pd.to_datetime(df['# created_date'],format="%d-%m-%Y")

<ipython-input-236-6a3d81c1bba4>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
daily_data=df_fin.groupby('created_date').agg({
    'id': 'first',
    'created_by': 'first',
    'transaction_branch_id': lambda x: x.mode()[0],
    'transaction_type_id': 'first',
    'amount_paid': 'sum',
    'Coupon_Discount': 'sum',
    'magazine_fee_paid': 'sum',
    'payable_amount': 'sum',
    'reading_fee_paid': 'sum',
    'reversed': 'first',
    'over_due_adjustment_amount': 'sum',
    'last_card_number': 'first',
    'primus_amount': 'sum',
    'adjustment_amount': 'sum',
    'security_deposit': 'sum',
    'Percentage_Share': 'sum',
    'subscription_id': 'first',
    'member_branch_id': 'first',
    'transaction': 'first',
    'Transaction_Month': 'first',
    'Transaction_Year': 'first',
    'created_name': 'first',
    'branch_name': 'first',
    'branch_type': 'first',
    'member_card': 'first',
    'Member_Name': 'first',
    'email': 'first',
    'Number_of_Months': lambda x: x.mode()[0],  # Most frequent value
    'display_name': 'first',
    'Renewal_Amount': 'sum',
    'Message_Status': 'first',
    'Membership_expiry_date': 'first',
    'User_Name': 'first'
}).reset_index()

daily_data.rename(columns={'created_date': 'ds', 'magazine_fee_paid': 'y'}, inplace=True)



In [ ]:
daily_data.head()

,ds,id,created_by,transaction_branch_id,transaction_type_id,amount_paid,Coupon_Discount,magazine_fee_paid,payable_amount,reading_fee_paid,...,branch_type,member_card,Member_Name,email,Number_of_Months,display_name,y,Message_Status,Membership_expiry_date,User_Name
0,2020-01-01,41532340,39988,810,2,110324,16216.000,1400.0000,124624,107284.0000,...,Company Owned,M1197057,APARNA SINGHI,caaparna3@gmail.com,3,Online Package,13530,0,13-09-2022,blrdrmembers
1,2020-01-02,41532409,1000,810,2,115690,12376.000,4446.6667,107484,107504.0037,...,Company Owned,M101322,HEEMANSHU D KHARATE,kheemanshu@gmail.com,1,Default package,18088,0,07-01-2023,blrdrmembers
2,2020-01-03,41532489,1000,810,2,156245,24420.000,3800.0000,170993,153793.3330,...,Company Owned,M084601,MANEESH,bavanamaneesh@gmail.com,3,Online Package,19694,0,30-01-2022,blrdrmembers
3,2020-01-04,41532586,1000,810,1,268741,38280.667,2903.3330,281913,255581.0030,...,Company Owned,M2200902,Jai,jai.pollayil@ansys.com,3,Online Package,28067,0,01-06-2021,blrdrmembers
4,2020-01-05,41532715,1000,810,2,271413,40020.000,8400.0000,275245,262180.0000,...,Company Owned,M1196580,SUDEEP KAUR BANSI,sudeep1110@gmail.com,3,Online Package,28116,0,03-05-2021,blrdrmembers


In [ ]:
#regressor for Number_of_Months:
#regressors=  ['reading_fee_paid', 'amount_paid', 'Coupon_Discount', 'payable_amount','transaction_type_id','transaction','member_branch_id']
#regressor for reading fee paid:
#regressors=  ['amount_paid', 'Number_of_Months', 'Coupon_Discount', 'payable_amount','id','Transaction_Year','reading_fee_adjustment_amount','transaction_type_id','magazine_fee_paid']
#regressor for magazine_fee_paid:
#regressors= ['amount_paid', 'reading_fee_paid', 'payable_amount','Coupon_Discount','subscription_id','id']
#regressor for coupon_Discount:
#regressors=['reading_fee_paid','Number_of_Months','amount_paid','payable_amount','id','subscription_id']

In [ ]:
model = Prophet()
for regressor in regressors:
    model.add_regressor(regressor)

In [ ]:
model = Prophet()
for regressor in regressors:
    # Check if the regressor column contains only numeric values
    if daily_data[regressor].dtype != 'object': # Only add regressors if they don't contain strings.
        model.add_regressor(regressor)
model.fit(daily_data[['ds', 'y'] + regressors])

TypeError: arg must be a list, tuple, 1-d array, or Series

In [ ]:
import pandas as pd

# Create and fit the model
model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)

# List to hold the valid numeric regressors
valid_regressors = []

# Check the regressors and add only numeric ones
for regressor in regressors:
    if pd.api.types.is_numeric_dtype(daily_data[regressor]):
        model.add_regressor(regressor)
        valid_regressors.append(regressor)

# Display which regressors are being used
print(f"Using regressors: {valid_regressors}")

# Fit the model using the selected columns
model.fit(daily_data[['ds', 'y'] + valid_regressors])


Using regressors: ['id', 'Transaction_Year', 'Transaction_Year', 'subscription_id', 'Transaction_Month']


TypeError: arg must be a list, tuple, 1-d array, or Series

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Function to calculate historical daily averages for each month in the period
def calculate_daily_averages(df, month, year, regressors):
    # Determine the range of years in the historical data
    min_year = df['ds'].dt.year.min()
    max_year = df['ds'].dt.year.max()

    # Calculate weights dynamically based on available years
    weights = {y: 1 / (year - y + 1) for y in range(min_year, max_year + 1)}

    daily_averages = {}
    for regressor in regressors:
        daily_values = []
        for day in range(1, 32):  # Assuming up to 31 days in a month
            day_values = []
            for y, weight in weights.items():
                day_data = df[(df['ds'].dt.month == month) &
                              (df['ds'].dt.year == y) &
                              (df['ds'].dt.day == day)]

                if not day_data.empty:
                    # Check if the column is numeric before calculating the mean
                    if pd.api.types.is_numeric_dtype(day_data[regressor]):
                        day_avg = day_data[regressor].mean() * weight
                        day_values.append(day_avg)

            if day_values:
                # Convert weights.values() to a list and then sum
                total_weight = sum(weights.values())
                weighted_average = np.sum(day_values) / total_weight
            else:
                weighted_average = np.nan  # Handle cases where there is no data for that day
            daily_values.append(weighted_average)

        daily_averages[regressor] = daily_values

    return daily_averages

# Historical Data (Assume you have it in 'daily_data')
historical_data = daily_data.copy()

# Predicting for the next 3 months starting from August 2024
start_date = datetime(2024, 8, 15)
end_date = datetime(2024, 10, 31)

# Regressors(Change it according to the variable you are forecasting for, all regressors are mentioned above)

regressors= [ 'date_create','Membership_expiry_date', 'id','membership_start_date','dailysales','subscription_id','Transaction_Month']




# Generate future DataFrame for the prediction period
future_dates = pd.date_range(start=start_date, end=end_date)
future_df = pd.DataFrame({'ds': future_dates})

# Loop through each month in the prediction period
for current_month in range(start_date.month, end_date.month + 1):
    # Calculate daily averages for the current month
    daily_averages = calculate_daily_averages(historical_data, current_month, start_date.year, regressors)

    # Assign daily averages to the corresponding dates in future_df
    month_mask = future_df['ds'].dt.month == current_month
    for regressor in regressors:
        future_df.loc[month_mask, regressor] = daily_averages[regressor][:sum(month_mask)]

# Optional: Adjust for day of the week effect
future_df['day_of_week'] = future_df['ds'].dt.dayofweek
weekend_factor = 0.9  # Example adjustment for weekends

for regressor in regressors:
    future_df.loc[future_df['day_of_week'] >= 5, regressor] *= weekend_factor

forecast = model.predict(future_df)

# Prepare details as a DataFrame
details_df = pd.DataFrame({
    'Detail': ['Forecasting Period', 'Regressors', 'Number of Days in future_df', 'Weekend Adjustment Factor'],
    'Value': [
        f"{start_date} to {end_date}",
        ", ".join(regressors),
        len(future_df),
        weekend_factor
    ]
})

combined_df = pd.concat([details_df, future_df, forecast], axis=1)


combined_file = 'forecast_data_5.csv'
combined_df.to_csv(combined_file, index=False)



In [ ]:
import matplotlib.pyplot as plt

# Use Prophet's built-in function to plot components
fig = model.plot_components(forecast)
fig.savefig('pplot_5.png')
plt.show()


In [ ]:
pip install -U kaleido


In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add forecast line
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast', line=dict(color='red')))

# Add uncertainty interval (optional)
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill=None, mode='lines', line_color='red', line=dict(dash='dash'), name='Upper Bound'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='lines', line_color='red', line=dict(dash='dash'), name='Lower Bound'))

# Update layout
fig.update_layout(title='Forecasted Values with Prophet',
                  xaxis_title='Date',
                  yaxis_title='Forecasted Value',
                  legend_title='Legend')
fig.show()
